In [12]:
!pip install torch
!pip install scikit-learn

  Using cached scipy-1.13.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.6 MB 1.7 MB/s eta 0:00:07
    --------------------------------------- 0.2/10.6 MB 2.0 MB/s eta 0:00:06
   - -------------------------------------- 0.3/10.6 MB 2.0 MB/s eta 0:00:06
   - -------------------------------------- 0.4/10.6 MB 2.2 MB/s eta 0:00:05
   - -------------------------------------- 0.5/10.6 MB 2.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.6/10.6 MB 2.2 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/10.6 MB 2.2 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/10.6 MB 2.3 MB/s eta 0:00:05
   --- ------------------------------------ 0.9/10.6 MB 2.3 MB/s eta 0:00:05
   --- ------------------------------------ 0.9/10.6 MB 2.2 MB/s eta 0:00:05
   --- ------------

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [14]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activations):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.activations = activations
        
        # Define layers
        layer_sizes = [input_size] + hidden_sizes + [output_size]
        self.layers = nn.ModuleList()
        
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
        
    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i < len(self.layers) - 1:
                activation = getattr(F, self.activations[i])
                x = activation(x)
        return torch.sigmoid(x)

In [15]:
# Создаем экземпляр MLP
input_size = 2
hidden_sizes = [64, 32]  # Размеры скрытых слоев
output_size = 1
activations = ['relu', 'relu']  # Функции активации для скрытых слоев

model = MLP(input_size, hidden_sizes, output_size, activations)
print(model)

# Пример применения модели к входным данным
sample_input = torch.randn(10, input_size)  # Пример входных данных
output = model(sample_input)
print(output)

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
)
tensor([[0.4870],
        [0.4684],
        [0.4778],
        [0.4637],
        [0.5072],
        [0.4668],
        [0.4754],
        [0.4798],
        [0.4918],
        [0.4926]], grad_fn=<SigmoidBackward0>)


In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Загружаем датасет Iris
iris = load_iris()
X = iris.data
y = (iris.target == 0).astype(int)  # Бинарная классификация: Iris Setosa или нет

# Разбиваем на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем Dataset для PyTorch
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Создаем DataLoader
batch_size = 32
train_dataset = IrisDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Определяем функцию обучения
def train(model, train_loader, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(1), targets)
            loss.backward()
            optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Инициализируем модель, оптимизатор и функцию потерь
input_size = X.shape[1]
hidden_sizes = [64, 32]
output_size = 1
activations = ['relu', 'relu']
model = MLP(input_size, hidden_sizes, output_size, activations)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Обучаем модель
train(model, train_loader, optimizer, criterion, epochs=100)


Epoch [10/100], Loss: 0.2923
Epoch [20/100], Loss: 0.0475
Epoch [30/100], Loss: 0.0138
Epoch [40/100], Loss: 0.0165
Epoch [50/100], Loss: 0.0061
Epoch [60/100], Loss: 0.0027
Epoch [70/100], Loss: 0.0035
Epoch [80/100], Loss: 0.0019
Epoch [90/100], Loss: 0.0010
Epoch [100/100], Loss: 0.0014
